## NCAA Early Applicant to NBA Picker
Today we are trying to see if we can determine whether given a USA NCAA player who's applied to the NBA, whether that person is likely to get drafted. We shall do this through a variety of techniques. Firstly we must generate our own unique dataset utilizing BigQuery's ncaa_mbb dataset. Our first issue with the ncaa_mbb dataset is we have no idea whether any of the applicants have applied to the nba. To solve this issue, we shall partition the data by adding another categorical feature to each player indicating whether this player applied to the nba. Specifically we will label whether a player is an early nba applicant, simply because early applicants seemed to be the only reliable form of data we could find regarding nba applicants. 

In [1]:
# Simple python libraries
import numpy as np
import matplotlib.pyplot as plt
import csv
from data_loader.DatasetsGenerator import getall
from data_loader.Draft import Draft
from data_loader.Applicants import Applicants
import os
import pandas as pd
# from data_loader import Draft, DatasetsGenerator
%matplotlib inline

# Generate all custom made datasets
getall()

# Create an nba drafted validation class
draft = Draft()

# Create a nba applicant validation class
applied = Applicants()

In [2]:
player_box = pd.read_csv("./data_loader/data/ncaa/player_box.csv")
player_info = pd.read_csv("./data_loader/data/ncaa/player_info.csv")

# Filter out unwanted values
player_box = player_box[player_box['player_id']!=-101]
player_box = player_box[~player_box['player_id'].isnull()]
player_box = player_box[player_box['season']!=None]
player_box = player_box[player_box['season']!=2018]

player_info = player_info[player_info['player_id']!=-101]
player_info = player_info[~player_info['player_id'].isnull()]
player_info = player_info[player_info['season']!=None]
player_info = player_info[player_info['season']!=2018]
player_info = player_info[~player_info['first_name'].isnull()]
player_info = player_info[~player_info['last_name'].isnull()]

# Combine the two datasets
player_box["player_id_AND_season"] = player_box["player_id"].map(int).map(str) + "_AND_" + player_box["season"].map(int).map(str) 
player_info["player_id_AND_season"] = player_info["player_id"].map(int).map(str) + "_AND_" + player_info["season"].map(int).map(str) 
player_stats = pd.merge(player_box, player_info, on='player_id_AND_season', how='outer')
player_stats = player_stats.dropna()

# Get only drafted players
player_stats = applied.join(player_stats)

# Add drafted categorical value
player_stats = draft.join(player_stats)
player_stats.to_csv('./data_loader/data/ncaa/player_stats.csv')


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(player_stats)

1382